# <center>[📝 Достаем текстовые эмбеддинги](https://stepik.org/lesson/825509/step/5?unit=829016)</center>

### Оглавление ноутбука

<img src='https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/images/bert.jpg' align="right" width="450" height="450" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">👁 Подключаемся к гугл-диску и GPU</a></li>
<li><a href="#c2">🗜 Импортируем библиотеки</a></li>
<li><a href="#c3">📩 Считываем данные</a></li>
<li><a href="#c4">🤗 Делаем предсказания языковой моделью</a></li>
<li><a href="#c5">🍢 Усредняем эмбеддинги</a></li>
<li><a href="#c6">🧵 Объединяем все в класс</a></li>
<li><a href="#c7">🗣 Советы по использованию языковых моделей</a>

</li></ul></font></p>

<div class="alert alert-info">
Вы уже наверняка слышали о базовых способах работы с текстами: tf-idf, наивный байес, word2vec. Однако большие предобученные языковые модели работают в тысячу раз лучше (думаю, вы уже слышали про ChatGpt). Поэтому в соревнованиях при работе с текстом обычно применяются именно они. И сегодня мы рассмотрим, как можно достать эмбеддинги текстовых признаков при помощи таких языковых моделей, чтобы потом использовать их в качестве признаков.

## 👁 Подключаемся к гугл-диску
<p id="c1"></p>   

<!-- <div class="alert alert-info"> -->
Языковые модели, которые мы будем рассматривать очень большие и поэтому для того, чтобы ими пользоваться и это не занимало вечность, нам нужно будет использовать графические ускорители (GPU). Для этого воспользовуемся сервисом [google colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true), который как раз предоставляет такие возможности. Первым делом подключимся к нашему гугл-диску и проверим, что GPU на месте.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 🗜 Импортируем библиотеки
<p id="c2"></p>   

In [ ]:
!pip install transformers sentencepiece sentence-transformers simpletransformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, default_data_collator, DebertaV2Tokenizer, PegasusForConditionalGeneration, PegasusTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.multioutput import MultiOutputRegressor
import torch

import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

## 📩 Считываем данные
<p id="c3"></p>   

In [ ]:
data_root = "https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/"
data = pd.read_csv(data_root + 'text_classification_train_only_text.csv')
data.head()

,text
0,Ледник Пасторури это цирковой ледник расположе...
1,Главные участники предстоящего Betokenoid 274 ...
2,Ttokenoid Btokenoid – карта с которой можно не...
3,В Сильверстоуне произошли крупные обновления а...
4,На протяжении более чем 30 лет Вестсайд являет...


In [ ]:
data['text'] = data['text'].astype(str)
data.shape

(7500, 1)

## 🤗 Делаем предсказания языковой моделью
<p id="c4"></p>   

<div class="alert alert-info">

Для того, чтобы проинференсить нам понадобятся две сущности: модель и токенайзер. Токенайзер будет предобрабатывать текст, чтобы его можно было подать в модель (разбивать на токены, считать attention_mask, обрезать лишний текст и тд), в то время как модель будет непосредственно считать сами эмбеддинги.

In [ ]:
model_name = 'sberbank-ai/ruBert-base'
max_len = 512

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).cuda()

In [ ]:
sentence = data['text'][0]
encoded_input = tokenizer([sentence], # список текстов, которые хотим прогнать
                          padding='max_length', # будем дополнять токены нулями до max_len
                          truncation=True, # если слишком много токенов, то будем удалять последние
                          max_length=max_len, # максмимальная длина текстов, с которой может работать модель
                          return_tensors='pt') # возвращать будем тензоры, чтобы было удобно работать
with torch.no_grad():
    hidden_state, cls_head = model(input_ids=encoded_input['input_ids'].cuda(), return_dict=False)
    # hidden_state - выходы модели для каждого токена, shape = (batch_size, max_len, emb_size)
    # cls_head - выход модели для токена [CLS], в котором аккумулируется информация о всем предложение, shape = (batch_size, emb_size)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
hidden_state.shape, cls_head.shape

(torch.Size([1, 512, 768]), torch.Size([1, 768]))

## 🍢 Усредняем эмбеддинги
<p id="c5"></p>   

<div class="alert alert-info">

Для того, чтобы посчитать общий эмбеддинг предложения можно либо взять информацию из `cls_head`, либо усреднить `hidden_state` по всем словам. Давайте посчитаем усредненный `hidden_state`.

In [ ]:
# attention_mask - содержит 1, если на этой позиции стоит осмысленный токен и 0, если это padding
attention_mask = encoded_input['attention_mask']
token_embeddings = hidden_state.detach().cpu()
attention_mask[0][:50], token_embeddings[0][0][:50]

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 tensor([ 0.3078,  0.4664,  0.7443,  0.8290, -0.4497,  0.3474, -0.6934,  0.3632,
          0.2229, -1.0116, -0.1834,  0.0818, -0.0139, -0.3390,  0.6213, -0.4389,
         -0.0878,  1.0532, -1.4599, -0.1087, -0.4355,  0.1824,  0.3825, -0.1280,
          0.5062, -0.0472,  0.3235,  0.3271, -0.5110,  0.2902,  0.3373, -0.0867,
         -0.5511,  1.3511,  0.0597,  1.2196, -0.1140, -0.3848,  0.1512,  0.1980,
         -1.9379,  0.0468,  0.9116, -0.3323,  1.1896,  0.0047, -0.0728,  0.0836,
          0.1591,  0.3771]))

In [ ]:
token_embeddings.shape, attention_mask.shape

(torch.Size([1, 512, 768]), torch.Size([1, 512]))

In [ ]:
attention_mask.unsqueeze(-1).shape

torch.Size([1, 512, 1])

In [ ]:
# растянем нашу маску так, чтобы ее размер совпадал с размером hidden_state
input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
input_mask_expanded.shape, sum_embeddings.shape

(torch.Size([1, 512, 768]), torch.Size([1, 768]))

In [ ]:
attention_mask.sum()

tensor(34)

In [ ]:
(sum_embeddings / attention_mask.sum()).shape

torch.Size([1, 768])

## 🧵 Объединяем все в класс
<p id="c6"></p>   

In [ ]:
class TextEmbeddings:
    def __init__(self, add_cls_embeddings=True, add_mean_embeddings=False):
        self.add_mean_embeddings = add_mean_embeddings
        self.add_cls_embeddings = add_cls_embeddings
        if add_cls_embeddings is False and add_mean_embeddings is False:
            raise 'Error: you should select at least one type of embeddings to be computed'

    def mean_pooling(self, hidden_state, attention_mask):
        """
        Возвращает усредненный с учетом attention_mask hidden_state.
        """
        token_embeddings = hidden_state.detach().cpu()
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        return sum_embeddings / attention_mask.sum()

    def extract_embeddings(self, texts, model_name, max_len):
        """
        Возвращает значения посчитанные данной моделью эмбеддинги для всех текстов из texts.
        """
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name).cuda()
        text_features = []
        for sentence in tqdm(texts):
            encoded_input = tokenizer([sentence],
                                      padding='max_length',
                                      truncation=True,
                                      max_length=max_len,
                                      return_tensors='pt')
            with torch.no_grad():
                hidden_state, cls_head = model(input_ids=encoded_input['input_ids'].cuda(), return_dict=False)
                sentence_embeddings = self.mean_pooling(hidden_state, encoded_input['attention_mask'])

            now_emb = []
            if self.add_cls_embeddings:
                now_emb.append(cls_head.detach().cpu().numpy().flatten())

            if self.add_mean_embeddings:
                now_emb.append(sentence_embeddings.detach().cpu().numpy().flatten())

            text_features.append(np.concatenate(now_emb, axis=0))
        return text_features

    def add_many_embeddings(self, df, text_col, models):
        """"
        Добавляет в качестве признаков эмбеддинги для колонки text_col.
        В качестве моделей и максимальных длинн используются models.
        """
        for model_name, max_len in models:
            print(model_name)
            text_features = self.extract_embeddings(df[text_col], model_name, max_len)
            text_features_df = pd.DataFrame(text_features, columns = [f'{model_name}_{text_col}_feature_{i}' for i in range(len(text_features[0]))])
            df = df.join(text_features_df)
            df.to_csv('transformers_text_features.csv', index=False)
            os.system('cp /content/transformers_text_features.csv /content/drive/MyDrive/datasets/transformers_text_features.csv')
        return df

In [ ]:
# Полный список поддерживаемых моделей можно найти на https://huggingface.co/models
models = [
          ('cointegrated/LaBSE-en-ru', 512),
        #   ('sberbank-ai/ruRoberta-large', 512),
        #   ('sberbank-ai/sbert_large_nlu_ru', 512),
        #   ('sberbank-ai/sbert_large_mt_nlu_ru', 512),
        #   ('sberbank-ai/ruBert-large', 512),
          ('sberbank-ai/ruBert-base', 512),
          ('cointegrated/rubert-tiny2', 2048),
          ('DeepPavlov/rubert-base-cased-conversational', 512),
        #   ('microsoft/mdeberta-v3-base', 512),
        #   ('vicgalle/xlm-roberta-large-xnli-anli', 512),
        #   ('MoritzLaurer/mDeBERTa-v3-base-mnli-xnli', 512),
        #   ('facebook/bart-large-mnli', 1024)
]

In [ ]:
text_embeddings = TextEmbeddings(True, True)
data = text_embeddings.add_many_embeddings(data, 'text', models)

cointegrated/LaBSE-en-ru


  0%|          | 0/7500 [00:00<?, ?it/s]

sberbank-ai/ruBert-base


  0%|          | 0/7500 [00:00<?, ?it/s]

cointegrated/rubert-tiny2


  0%|          | 0/7500 [00:00<?, ?it/s]

DeepPavlov/rubert-base-cased-conversational


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/7500 [00:00<?, ?it/s]

In [ ]:
data.shape

In [ ]:
data.head()

## 🗣 Советы по использованию языковых моделей
<p id="c7"></p>   

<div class="alert alert-info">
Напоследок, несколько советов по работе с текстами.
    
**Когда применять языковые модели:**
1) Тексты не очень большие (в идеале до 512 токенов)
2) Вы хотите опираться на смысл текста, а не на ключевые слова
3) Есть GPU ресурсы и от вас не требуют мгновенной скорости работы
    
**Как выбрать какие модели использовать:**
1) Лучше всего использовать модели, предобученные на вашем домене, если такие есть
2) Обратите внимание, что ваша модель работает с необходимым вам языком
3) Если очень хочется, то тексты можно перевести на английский (см. урок про парсинг) и использовать модели уже для переведнных текстов
4) Модели можно дообучать на вашей задаче
5) Можно доставать эмбеддинги и использовать их в качестве признаков, а можно просто предсказывать таргет с помощью языковой модели
6) Если есть время и мощности, то не пренебрегайте стэкингом разных языковых моделей